In [308]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
proj_dir = os.path.abspath('../..')

if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))

import numpy as np
import scipy.stats as stats
import pandas as pd
from scipy.spatial import distance
from random import random
from sklearn.cluster import SpectralBiclustering
import itertools

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from matplotlib import colors

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import blockworld_helpers as utils
import drawing_utils as drawing
import importlib
import scoring
import rda

from pyvis.network import Network
import networkx as nx

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('../..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(analysis_dir,'results')
stim_dir = os.path.join(proj_dir,'stimuli')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
iteration_name = 'Exp2Pilot3'
num_trials = 24 #for sanity checks

In [4]:
# Data already compiled into dataframes in CogSci 2020 Dataframe Generator

# trial_end data
trial_path = os.path.join(csv_dir,'block_silhouette_{}_good.csv'.format(iteration_name))
df = pd.read_csv(trial_path)

# # initial_block data
initial_path = os.path.join(csv_dir,'block_silhouette_initial_{}_good.csv'.format(iteration_name))
dfi = pd.read_csv(initial_path)

# # settled_block data
settled_path = os.path.join(csv_dir,'block_silhouette_settled_{}_good.csv'.format(iteration_name))
dfs = pd.read_csv(settled_path)

# # Sanity Check- same participants in each dataset.
df_participants = df.gameID.unique()
dfs_participants = dfs.gameID.unique()
assert Counter(df_participants) == Counter(dfs_participants)

n_before_outliers = len(df_participants)
print(str(n_before_outliers) + ' participants total')

49 participants total


In [6]:
def convert_to_str(flat_world):
    s = [str(i) for i in list(flat_world)] 
    res = "".join(s)
    return res

In [7]:
targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())
reps = np.sort(df['repetition'].unique())

dfi['usableDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: 1+(-1)*np.array(ast.literal_eval(a)))
dfi['flatDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: (1+(-1)*np.array(ast.literal_eval(a))).flatten())

dfic = dfi[dfi.condition=='repeated']
dfic = dfic[['targetName','gameID','blockNum','repetition','flatDiscreteWorld']]
dfic['flatDiscreteWorldStr'] = dfic['flatDiscreteWorld'].apply(convert_to_str)

max_actions = dfic['blockNum'].max()

In [313]:
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
net.barnes_hut()

In [314]:
# need an easy way of representing thickness of lines between two nodes

In [320]:
class Node:
    '''
    Represents a world state (as defined by discrete bitmap representation of blocks present, 
        as opposed to individual block dimensions and locations).
    Each Node can have multiple children.
    '''
    
    def __init__(self, flat_world):
        self.out_edges = [] #list of edges (child, ppt, rep)
        self.flat_world = flat_world 
        self.flat_world_str = convert_to_str(flat_world)
        self.world_size = np.sum(flat_world)
        
        net.add_node(self.flat_world_str, label=self.flat_world_str)
        
    def __str__(self):
        return('Node of size ' + str(self.world_size))
    
    def __repr__(self):
        return('Node of size ' + str(self.world_size))

    #def get_ppts(self):
        #self.out_edges.m
        
    def add_child(self, node, ppt, rep, block_num):
        self.out_edges.append(Edge(self, node, ppt, rep, block_num))

In [321]:
class Edge:
    '''
    Edges contain child nodes, as well as the (ppt, repetition) pair that 
        uniquely defines a building path in the experiment (for one structure)
    '''
    def __init__(self, source, target, ppt, rep, block_num):
        self.source = source
        self.target = target
        self.ppt = ppt
        self.rep = rep
        self.block_num = block_num
        print('created edge: ' + str(self))
        
        net.add_edge(self.source.flat_world_str, self.target.flat_world_str)
        
    def __str__(self):
        return('Block ' + str(self.block_num) + ', ppt: ' + self.ppt + ', rep: ' + str(self.rep))
    
    def __repr__(self):
        return('Block ' + str(self.block_num) + ', ppt: ' + self.ppt + ', rep: ' + str(self.rep))


In [322]:
class WorldLayer:
    '''
    Height in a tree is set by the sum of 'on' squares in the bitmap representation of the world.
    WorldLayer contains a dictionary that maps a flatdiscreteworld to a Node object
    '''
    def __init__(self, num_squares):
        self.num_squares = num_squares
        self.nodes = {}
        
    def __str__(self):
        return(str(len(self.nodes)) + ' nodes in layer')
    
    def __repr__(self):
        return(str(len(self.nodes)) + ' nodes in layer')
    
    def get_world_node(self, flat_world):
        '''
        add world state into layer
        if already exists, then returns the existing node.
        otherwise, creates new node and returns it
        
        '''
        flat_world_str = convert_to_str(flat_world)
        if flat_world_str in self.nodes:
            existing_node = self.nodes[flat_world_str]
            return existing_node
        else:
            new_node = Node(flat_world)
            self.nodes[flat_world_str] = new_node
            return new_node

In [323]:
class BuildTree:
    def __init__(self, target_name):
        self.target_name = target_name
        self.world_layers = {} #dict.fromkeys(range(0, 80, 2))
        self.world_layers[0] = WorldLayer(0) # layers to allow quick indexing of world state
        
        # Add layer for every multiple of two, or add manually?
        
        self.root = self.world_layers[0].get_world_node(np.zeros(13*18).astype(int)) # start with one node with empty world
        self.prev_node = self.root
        
        
    def __str__(self):
        return('Tree for ' + self.target_name + ': \n' + str(self.root) + '\nLayers: ' + str(self.world_layers))
    
    def add_block_placement(self, discrete_world, discrete_world_str, ppt, rep, block_num):
        # fetch correct node (or create new one)
        world_squares = np.sum(discrete_world)
        if not(world_squares in self.world_layers): #if layer doesn't exist, add layer
            self.world_layers[world_squares] = WorldLayer(world_squares)
            
        # get node for this world state (if exists)
        new_node = self.world_layers[world_squares].get_world_node(discrete_world)
        
        # link previous nodes to current node
        self.prev_node.add_child(new_node, ppt ,rep, block_num)
        self.prev_node = new_node
        
    def reset_prev_node(self):
        '''
        Reset pointer to previous node
        '''
        self.prev_node = self.root
        
        #print(discrete_world, discrete_world_str, ppt, rep)
        
    def add_build_path(self, df):
        '''
        adds series of block placements from dataframe (for one build sequence) to the tree
        '''
        df.apply(lambda r: self.add_block_placement(r.flatDiscreteWorld,
                                        r.flatDiscreteWorldStr,
                                        r.gameID,
                                        r.repetition,
                                        r.blockNum), axis=1)
        self.reset_prev_node()

In [324]:
t = BuildTree(targets[0])

In [325]:
a = dfic[(dfic.targetName==targets[0]) & (dfic.repetition==0)]
a = a.groupby('gameID')
a.apply(lambda g: t.add_build_path(g))

created edge: Block 1, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 2, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 3, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 4, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 5, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 6, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 7, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 8, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 9, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 10, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 11, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 12, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0
created edge: Block 1, ppt: 0443-fd078367-c19d-495d-84d5-3519

""


In [327]:
net.show("tree_graph.html")

In [287]:
a = dfic[(dfic.targetName==targets[0]) & (dfic.gameID==ppts[8]) & (dfic.repetition==0)]
t.add_build_path(a)

8
created edge: Block 1, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
12
created edge: Block 2, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
20
created edge: Block 3, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
28
created edge: Block 4, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
36
created edge: Block 5, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
40
created edge: Block 6, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
44
created edge: Block 7, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
46
created edge: Block 8, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
50
created edge: Block 9, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
54
created edge: Block 10, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
56
created edge: Block 11, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
58
created edge: Block 12, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0
60
created edge: Block 13,

In [291]:
print(t)

Tree for hand_selected_004: 
Node of size 0
Layers: {0: 1 nodes in layer, 64: 2 nodes in layer, 66: 1 nodes in layer, 4: 3 nodes in layer, 6: 1 nodes in layer, 8: 4 nodes in layer, 10: 3 nodes in layer, 12: 6 nodes in layer, 14: 5 nodes in layer, 16: 9 nodes in layer, 18: 2 nodes in layer, 20: 10 nodes in layer, 22: 10 nodes in layer, 24: 13 nodes in layer, 26: 2 nodes in layer, 28: 12 nodes in layer, 30: 5 nodes in layer, 32: 11 nodes in layer, 34: 7 nodes in layer, 36: 13 nodes in layer, 38: 7 nodes in layer, 40: 10 nodes in layer, 42: 5 nodes in layer, 44: 10 nodes in layer, 46: 6 nodes in layer, 48: 7 nodes in layer, 50: 4 nodes in layer, 52: 4 nodes in layer, 54: 1 nodes in layer, 56: 4 nodes in layer, 58: 2 nodes in layer, 60: 1 nodes in layer, 62: 1 nodes in layer}


In [292]:
t.root.out_edges

[Block 1, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0,
 Block 1, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0,
 Block 1, ppt: 0477-2df104ca-e405-4c6c-a35d-eb6b52cdde96, rep: 0,
 Block 1, ppt: 0652-88ebf36e-0780-4434-a9c7-969af27c2636, rep: 0,
 Block 1, ppt: 0696-c7f6bc88-04be-490d-9bf4-0e0092946d86, rep: 0,
 Block 1, ppt: 0750-6e9daa66-cf41-4b5f-8626-8cac81c5597b, rep: 0,
 Block 1, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0,
 Block 1, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0,
 Block 1, ppt: 0443-fd078367-c19d-495d-84d5-3519a281435d, rep: 0,
 Block 1, ppt: 0477-2df104ca-e405-4c6c-a35d-eb6b52cdde96, rep: 0,
 Block 1, ppt: 0652-88ebf36e-0780-4434-a9c7-969af27c2636, rep: 0,
 Block 1, ppt: 0696-c7f6bc88-04be-490d-9bf4-0e0092946d86, rep: 0,
 Block 1, ppt: 0750-6e9daa66-cf41-4b5f-8626-8cac81c5597b, rep: 0,
 Block 1, ppt: 0976-bad77500-c02a-44f0-bb87-5dd2768e977c, rep: 0,
 Block 1, ppt: 1106-c12ab2b8-d38b-4e8f-9244-5adfbadd1093, rep: 0,
 Block 1, 

In [293]:
t.world_layers

{0: 1 nodes in layer,
 4: 3 nodes in layer,
 6: 1 nodes in layer,
 8: 4 nodes in layer,
 10: 3 nodes in layer,
 12: 6 nodes in layer,
 14: 5 nodes in layer,
 16: 9 nodes in layer,
 18: 2 nodes in layer,
 20: 10 nodes in layer,
 22: 10 nodes in layer,
 24: 13 nodes in layer,
 26: 2 nodes in layer,
 28: 12 nodes in layer,
 30: 5 nodes in layer,
 32: 11 nodes in layer,
 34: 7 nodes in layer,
 36: 13 nodes in layer,
 38: 7 nodes in layer,
 40: 10 nodes in layer,
 42: 5 nodes in layer,
 44: 10 nodes in layer,
 46: 6 nodes in layer,
 48: 7 nodes in layer,
 50: 4 nodes in layer,
 52: 4 nodes in layer,
 54: 1 nodes in layer,
 56: 4 nodes in layer,
 58: 2 nodes in layer,
 60: 1 nodes in layer,
 62: 1 nodes in layer,
 64: 2 nodes in layer,
 66: 1 nodes in layer}

In [224]:
a = dfic[(dfic.targetName==targets[0]) & (dfic.gameID==ppts[1]) & (dfic.repetition==0)]
a

,targetName,gameID,blockNum,repetition,flatDiscreteWorld,flatDiscreteWorldStr
171,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
172,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,2,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
173,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
174,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,4,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
175,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,5,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
176,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,6,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
177,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
178,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,8,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
179,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,9,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
180,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,10,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...


In [312]:
nxg = nx.complete_graph(10)
G = Network()
G.from_nx(nxg)
#net.enable_physics(True)
net.show("mygraph.html")

In [311]:
net.

<class 'pyvis.network.Network'> |N|=0 |E|=0